# ナイーブベイズによる文書判別における尤度確認

scikit-learnで文書判別を行なう際に、判別に影響する単語の尤度がどれくらいであるか確認する。
また、scikit-learnの処理中における変数内のデータを出力することにより、実装の理解を深める。




In [1]:
import chazutsu

r = chazutsu.datasets.MovieReview.polarity().download(force=True, test_size=0.0)

## CountVectorizer.fit_transform

訓練データである文書集合から単語を抽出し、ボキャブラリ(単語一覧)を行列として返す。
行列は疎行列(スパース行列)であり、scipyのcsc_matrix型で格納されている。


In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# BoWに変換する
count_vect = CountVectorizer(token_pattern=r'[A-Za-z_]+')
X_train_counts = count_vect.fit_transform(r.data().review)

# 疎行列(csr-matrix)となっている。内容確認する。
f = open('data/X_train_counts.txt', 'w')

# dataは文書における各単語の出現回数
for idx in range(len(X_train_counts.data)):
    f.writelines(str(X_train_counts.data[idx]) + " ")
f.writelines(str("\n"))

# indicesは出現数が0ではない何列目の要素(単語)か
for idx in range(len(X_train_counts.indices)):
    f.writelines(str(X_train_counts.indices[idx]) + " ")
f.writelines(str("\n"))

# indptrはどこまで同一の行(文書)か
for idx in range(len(X_train_counts.indptr)):
    f.writelines(str(X_train_counts.indptr[idx]) + " ")
f.writelines(str("\n"))

f.close()

今回の訓練データ全体の単語は39204種類。

In [3]:
len(count_vect.vocabulary_)

39204

## 学習

1つ目の文書にスポットを当てて確認する。
1つ目の文書に出てくる単語の種類と出現数を確認。


In [4]:
# dictにおける値からkeyを取得する関数。vocaburaly_から単語idを用いて単語名を取得するのに用いる
def dict_value_to_key(items, in_val):
    ret_key = ""
    for key, value in items:
        if value == in_val:
            ret_key = key
            break
    return ret_key


# 1つ目の文書の単語の種類と出現数を書き出し
f = open('data/first_document_vocabulary.txt', 'w')

for ptr in range(X_train_counts.indptr[1]):    
    voc = dict_value_to_key(count_vect.vocabulary_.items(), X_train_counts.indices[ptr])

    # 出力形式:単語id 単語名 出現回数を出力
    f.writelines(str(X_train_counts.indices[ptr]) + " " + str(voc) + " " + str(X_train_counts.data[ptr]) + "\n")

f.close()

ナイーブベイズ分類を実行。学習した単語の尤度を確認。


In [7]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, r.data().polarity)

# 単語毎に各クラスにおけるの尤度を取得
f = open('data/feature_log_prob_.txt', 'w')
for i in range(len(clf.feature_log_prob_[0])):
    voc = dict_value_to_key(count_vect.vocabulary_.items(), i)

    # 出力形式:単語名 ネガティブクラスにおける各単語の条件付き確率 ポジティブクラスにおける各単語の条件付き確率 
    f.writelines(voc + " " + str(clf.feature_log_prob_[0][i]) + " " + str(clf.feature_log_prob_[1][i]) + "\n") 
f.close()

## 文書分類

学習した分類器 clf を用いて文書分類を行なうには以下のように実行する。

`predicted = clf.predict(X_new_tfidf)`

ここからpredictの処理を見ていく。predict内の処理は以下の通り。

```
        jll = self._joint_log_likelihood(X)
        return self.classes_[np.argmax(jll, axis=1)]
```

_joint_log_likelihoodで各カテゴリの事後確率を算出し、そのうち最も高いクラスを推定クラスとする。(MAP推定)


_joint_log_likelihoodの処理は以下の通り。

```
        check_is_fitted(self, "classes_")

        X = check_array(X, accept_sparse='csr')
        return (safe_sparse_dot(X, self.feature_log_prob_.T) +
                self.class_log_prior_)
```

事後確率を算出している処理を確認する。

`safe_sparse_dot(X, self.feature_log_prob_.T) + self.class_log_prior_`


In [8]:
# 尤度(カテゴリ×単語)の転置行列
print(clf.feature_log_prob_.T)

clf.feature_log_prob_.T.shape

[[-10.07997067 -12.41784378]
 [-12.719028   -13.51645606]
 [-12.719028   -13.51645606]
 ..., 
 [-12.719028   -12.82330888]
 [-13.41217518 -12.41784378]
 [-12.719028   -13.51645606]]


(39204, 2)

テストデータ文書の出現単語と学習していた単語毎の尤度を用いて行列積を求める。
求めた結果が推定時の尤度となる。


In [9]:
# safe_sparse_dot(X_train_counts, clf.feature_log_prob_.T)

from sklearn.utils.extmath import safe_sparse_dot
from scipy.sparse import issparse


def ssd(a, b, dense_output=False):
    if issparse(a) or issparse(b):
        ret = a * b
        if dense_output and hasattr(ret, "toarray"):
            ret = ret.toarray()
        return ret
    
    else:
        return fast_dot(a, b)

safe_sparse_dot_value = ssd(X_train_counts, clf.feature_log_prob_.T)

safe_sparse_dot_value.shape

(2000, 2)

各クラスの事前確率がclass_log_prior_に格納されている。

今回はpositive,negativeともに1000文書ずつ、合計2000文書を訓練データとして与えている。
それらの確率をlogで算出するとその通りになっていることが分かる。


In [10]:
import math
print("class_log_prior_: " + str(clf.class_log_prior_))

print(math.log(0.5))

class_log_prior_: [-0.69314718 -0.69314718]
-0.6931471805599453


尤度と事前確率を加算(いずれも対数変換しているので)を行なうことで、事後確率が算出される。
1番目の文書を見ると、ネガティブは「-3937.36437245」、ポジティブは「-3904.99122672」なのでポジティブと推定される。


In [11]:
score = safe_sparse_dot_value + clf.class_log_prior_
score

array([[ -3937.36437245,  -3904.99122672],
       [ -4514.96929083,  -4412.54616969],
       [ -3315.38618094,  -3272.76981737],
       ..., 
       [ -3624.98324263,  -3694.10381655],
       [ -7778.33253256,  -7881.31132756],
       [-10481.45229072, -10453.49552954]])

実際にpredictを実行してみると合致することが確認できる。

In [12]:
predicted = clf.predict(X_train_counts)
predicted

array([1, 1, 1, ..., 0, 0, 1])